In [2]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import *
# Importing standard Qiskit libraries and configuring account
from qiskit_ibm_runtime import QiskitRuntimeService, Options
#import qiskit_ibm_runtime 
#from qiskit.primitives import BackendSampler
#from qiskit.providers.fake_provider import FakeMumbaiV2
from qiskit_aer.primitives import Sampler
#from qiskit.utils import algorithm_globals
#Load feature maps
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.circuit.library import ZZFeatureMap
from qiskit_algorithms.state_fidelities import ComputeUncompute
#from Kernels.src.ComputeUncompute import ComputeUncompute
#from Kernels.src.ComputeUncompute_2 import ComputeUncompute_2
from qiskit_machine_learning.kernels import FidelityQuantumKernel

import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [3]:
from qiskit_ibm_runtime.fake_provider import FakeMumbai

In [7]:
backend = FakeMumbai()

/tmp/ipykernel_174933/4105265394.py:1: DeprecationWarning: V1 fake backends are deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use V2 fake backends instead.
  backend = FakeMumbai()


In [8]:
plot_coupling_map(backend)

MissingOptionalLibraryError: "The 'Graphviz' library is required to use 'plot_coupling_map'.  To install, follow the instructions at https://graphviz.org/download/. Qiskit needs the Graphviz binaries, which the 'graphviz' package on pip does not install. You must install the actual Graphviz software."

In [2]:
########################QUANTUM SESSION#########################################################
# Loading your IBM Quantum account(s)
print('Loading IBM Quantum account')
# Loading your IBM Quantum account(s)
service=QiskitRuntimeService(channel="ibm_quantum",token="95e70510a429e03c7cc4ce4c5c979c794135dbacffbebe5325dcd8b627cfe42c9f61e91945ba1adea992f81849c74cc5e0ed1ab93977db5084dd26a28fd25dd8")
print('selecting backend')
#backend = service.get_backend('ibmq_qasm_simulator')
backend_or=service.least_busy(operational=True, simulator=False)
#backend=FakeMumbaiV2()
noise_model = NoiseModel.from_backend(backend_or)
backend=AerSimulator.from_backend(backend_or)

print(backend)
target = backend.target
coupling_map = target.build_coupling_map()

Loading IBM Quantum account
selecting backend


In [5]:
print('FT map instance')
#Instance FTMAP
n_qubits=4
reps=1
ft_map = ZZFeatureMap(feature_dimension=n_qubits, reps=1)

#transpile circuit
print('transpile circuit')
pm=generate_preset_pass_manager(backend=backend,optimization_level=1)
print(type(pm))
ft_map_t_qs = pm.run(ft_map)

FT map instance
transpile circuit
<class 'qiskit.transpiler.passmanager.StagedPassManager'>


In [6]:
#Generate sample data
X_train=np.random.rand(10,4)

scaler = MinMaxScaler(feature_range=(0, 1*np.pi))
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [12]:
"""
#Set primitive sampler options
options = Options()
#Error mitigation level (resilience_level)
options.resilience_level = 1
#Optimization level
options.optimization_level = 3
#Number of shots
options.execution.shots = 2000
#Skip translation since the circuit is already transpiled
options.skip_transpilation= False


"""


# Create a quantum kernel based on the transpiled feature map
#Set Primitive sampler
#sampler = qiskit_ibm_runtime.Sampler(backend=backend)
sampler = Sampler(backend_options={"noise_model": noise_model, "shots": 2000, "seed_simulator":42})


In [13]:
print('Compose circuit with already transposed ftmap and run it')
ft_map_t_p=ft_map_t_qs.assign_parameters(X_train_scaled[0])
circuit=ft_map_t_qs.compose(ft_map_t_qs.inverse())
circuit.measure_active()
#run circuit
job = sampler.run(circuit,parameter_values=[np.random.rand(4)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

Compose circuit with already transposed ftmap and run it
>>> Job ID: 4e23234c-1b26-4bc7-b490-324642bf14e6
>>> Job Status: JobStatus.RUNNING


In [8]:
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

>>> Job ID: b8c7d38c-a9aa-405c-9c2a-8894eef7d174
>>> Job Status: JobStatus.RUNNING


In [9]:
print('Case 2: Compose and then transpose full circuit')
circuit_2=ft_map.compose(ft_map.inverse())
circuit_2.measure_active()
#transpose
isa_circuit_2=pm.run(circuit_2)
job_2 = sampler.run(isa_circuit_2,parameter_values=[np.random.rand(4)])
print(f">>> Job ID: {job_2.job_id()}")
print(f">>> Job Status: {job_2.status()}")


Case 2: Compose and then transpose full circuit


>>> Job ID: ba925b51-712f-4246-9f29-7e2485444ca6
>>> Job Status: JobStatus.RUNNING


In [10]:
print(f">>> Job ID: {job_2.job_id()}")
print(f">>> Job Status: {job_2.status()}")

>>> Job ID: ba925b51-712f-4246-9f29-7e2485444ca6
>>> Job Status: JobStatus.RUNNING


### Run as fidelity kernel

In [14]:
print('Run as fidelity kernel normal way')
fidelity = ComputeUncompute(sampler=sampler)
#run circuit using fidelity
test=fidelity._run(ft_map_t_qs,ft_map_t_qs,X_train_scaled[0],X_train_scaled[1])


Run as fidelity kernel normal way


In [9]:
#Set fidelity
print('Run as a fidelity Kernel but with inside transpilation like case 2')
fidelity_2 = ComputeUncompute_2(sampler=sampler,pm=pm)
test_2=fidelity_2._run(ft_map,ft_map,X_train_scaled[0],X_train_scaled[1])



Run as a fidelity Kernel but with inside transpilation like case 2


In [ ]:
qk=FidelityQuantumKernel(feature_map=ft_map_t_qs,fidelity=fidelity)
qk.evaluate(X_train_scaled[:10])